In [26]:
# CTD Calibration Parser for xmlcon file
import sys, os, csv, fnmatch
import datetime
import time
import xml.etree.ElementTree as et
from zipfile import ZipFile

class CTDCalibration(object):
    """
    Create an object for a CTD in order to load
    calibration.
    
    Attributes:
        Initializes the map of CTD calibration coeff
        names to the names accepted by OOINet.
    Functions:
        
    """
    # Class that stores calibration values for CTDs.
    # \param self
    def __init__(self):
        self.coefficient_name_map = {
            'TA0': 'CC_a0',
            'TA1': 'CC_a1',
            'TA2': 'CC_a2',
            'TA3': 'CC_a3',
            'CPCOR': 'CC_cpcor',
            'CTCOR': 'CC_ctcor',
            'CG': 'CC_g',
            'CH': 'CC_h',
            'CI': 'CC_i',
            'CJ': 'CC_j',
            'G': 'CC_g',
            'H': 'CC_h',
            'I': 'CC_i',
            'J': 'CC_j',
            'PA0': 'CC_pa0',
            'PA1': 'CC_pa1',
            'PA2': 'CC_pa2',
            'PTEMPA0': 'CC_ptempa0',
            'PTEMPA1': 'CC_ptempa1',
            'PTEMPA2': 'CC_ptempa2',
            'PTCA0': 'CC_ptca0',
            'PTCA1': 'CC_ptca1',
            'PTCA2': 'CC_ptca2',
            'PTCB0': 'CC_ptcb0',
            'PTCB1': 'CC_ptcb1',
            'PTCB2': 'CC_ptcb2',
            # additional types for series O
            'C1': 'CC_C1',
            'C2': 'CC_C2',
            'C3': 'CC_C3',
            'D1': 'CC_D1',
            'D2': 'CC_D2',
            'T1': 'CC_T1',
            'T2': 'CC_T2',
            'T3': 'CC_T3',
            'T4': 'CC_T4',
            'T5': 'CC_T5',
        }

        self.o2_coefficients_map = {
            'A': 'CC_residual_temperature_correction_factor_a',
            'B': 'CC_residual_temperature_correction_factor_b',
            'C': 'CC_residual_temperature_correction_factor_c',
            'E': 'CC_residual_temperature_correction_factor_e',
            'SOC': 'CC_oxygen_signal_slope',
            'OFFSET': 'CC_frequency_offset'
        }
        
        # Initialize dictionary with calibration coefficient names and values
        self.coefficients = {}
        self.asset_tracking_number = None
        self.serial = None
        self.date = None
        self.type = 'CTD'
        
        
    def load_cals(self, filepath):
        """
        Function to load the CTD calibration values into a CTD object.
        Searches the given filepath for first a .cal file, followed by
        a .xmlcon file. Raises an error if neither can be located in 
        the given filepath (additional compatability with pdfs under dev)
        Will also load directly from a zip file without extraction.
        
        Args:
            Path to the calibration file. Searches first for a .cal file,
            followed by a .xmlcon file. If a .cal file is found, calls a 
            load_cal function, else if .xmlcon file calls a load_xml function.
        Raises:
            Exception: If no .cal or .xmlcon file found
        Returns:
            Populates CTD object's coefficients dictionary with OOINet coeff
            name and associated value. Also fills in CTD object's serial num,
            date, and asset tracking number.
            
        """
        # Check if a zip file, and if so 
        if filepath.endswith('.zip'):
            with ZipFile(filepath) as zfile:
                if any('.cal' in x for x in zfile.namelist()):
                    findex, *ignore = [(i,x) for i,x in enumerate(zfile.namelist()) if '.cal' in x][0]
                    filename = zfile.namelist()[findex]
                    data = zfile.read(filename).decode('ASCII')
                    self.read_cal(data)
                elif any('.xmlcon' in x for x in zfile.namelist()):
                    findex, *ignore = [(i,x) for i,x in enumerate(zfile.namelist()) if '.xmlcon' in x][0]
                    filename = zfile.namelist()[findex]
                    data = et.parse(filename)
                    self.read_xml(data)
                else:
                    raise Exception('No calibration (.cal) or xmlcon file found.')
                    
        elif filepath.endswith('.xmlcon'):
            with open(filepath) as filename:
                data = et.parse(filepath)
                self.read_xml(data)
        
        elif filepath.endswith('.cal'):
            with open(filepath) as filename:
                data = filename.read()
                if type(data) is 'byte' or 'bytearray':
                    data = data.decode('ASCII')
                self.read_cal(data)
                
        else:
            raise Exception('No calibration or xmlcon files found.')
        
    def read_xml(self, data):
        """
        Loads xmlcon type data. Requires a parsed open datafile in
        order to read and load the calibration values.
        
        Args:
            self - the CTD object
            data - a parsed open datafile which will be read to load
            the xmlcon data into 
        Returns:
            A populated CTD object's dictionary with coeff names and
            associated values from the xmlcon file.        
        """
        
        Tflag = False
        O2flag = False
        coeffs = {}
        
        for child in data.iter():
            key = child.tag.upper()
            value = child.text.upper()
        
            # Do a couple of checks for type of CTD and flag for presence of
            # Oxygen sensor, Type (16+ vs 37)
            if key == 'NAME':
                if 'SEACAT' in value:
                    S16flag = True
                else:
                    S16flag = False
                
            if key == 'OXYGENSENSOR':
                O2flag = True
        
            if key == 'SERIALNUMBER':
                if self.serial is None and value is not None:
                    if S16flag == True:
                        self.serial = '16-' + value
                    else:
                        self.serial = value
        
            if key == 'CALIBRATIONDATE':
                if self.date is None and value is not None:
                    self.date = datetime.datetime.strptime(value, '%d-%b-%y').strftime('%Y%m%d')
            
            # Have to rename the temperature keys to 'T'+key because fuck it, nothing is straightforward
            if key == 'TEMPERATURESENSOR':
                Tflag = True
            elif 'SENSOR' in key and Tflag == True:
                Tflag = False
            else:
                pass
        
            if Tflag == True:
                key = 'T'+key
        
            # Now, can update a dictionary to store key->value pairs of coefficients from the xmlcon file    
            coeffs.update({key:value})
            
        for key,name in self.coefficient_name_map.items():
            try:
                self.coefficients[name] = coeffs[key]
            except:
                pass
    
        if O2flag == True:
            for key,name in self.o2_coefficients_map.items():
                try:
                    self.coefficients[name] = coeffs[key]
                except:
                    pass
         
        
    def read_cal(self, data):
        """
        Reads in the calibration coefficients from the vendor supplied
        .cal file.
        
        Args:
            self - the CTD object
            data - an opened, read cal file that has been interpreted
            into ASCII.
        Returns:
            A populated CTD object's dictionary with coeff names and
            associated values from the cal file. 
        """
        for line in data.splitlines():
            key, value = line.replace(" ","").split('=')

            if key == 'INSTRUMENT_TYPE' and value == 'SEACATPLUS':
                self.serial = '16-'

            if key == 'SERIALNO':
                self.serial = self.serial + value
    
            if key == 'CCALDATE':
                self.date = datetime.datetime.strptime(value, '%d-%b-%y').strftime('%Y%m%d')

            name = self.coefficient_name_map.get(key)
            if not name or name is None:
                continue
            else:
                self.coefficients[name] = value
                
    def write_csv(self, filepath):
        """
        Function which takes in a CTD object with loaded coefficient values
        and write them to a csv file in the file path in the correct.
        
        
        """
        
    def get_uid_lookup(self, pattern, path):
        """
        Function which gets a csv file with mapping from
        instrument manufacturer's serial numbers to the
        OOINet Asset UIDs
        
        Args:
            pattern - the unix pattern to match for finding
            the lookup file
            path - parent directory. Function will search all
            child directories for the given pattern to match
        Returns:
            the relative file path and file name of the lookup
            table
        """
        for root, dirs, files in os.walk(path):
            for name in files:
                if fnmatch.fnmatch(name.lower(), pattern):
                    return os.path.join(root, name)
                
    def get_uid(self, pattern, path):
        
        lookup_file = self.get_uid_lookup(pattern, path)
        lookup = {}
        
        with open(lookup_file) as csvfile:
            reader = csv.DictReader(csvfile, delimiter=',')
            for row in reader:
                key = row['Serial Number']
                value = row['ASSET_UID']
                lookup[key] = value
                
        self.asset_tracking_number = lookup[self.serial]
        

In [27]:
#filepath = "C:/Users/areed/Documents/Project_Files/Records/Instrument_Records/Sea-Bird/CTDBP-O_SBE_16PlusV2_SN_7249_Calibration_Files_2012-1-15.zip"
filepath = "/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/Sea-Bird/CTDBP-O_SBE_16PlusV2_SN_7249_Calibration_Files_2012-1-15.zip"

In [28]:
CTD = CTDCalibration()

In [29]:
CTD.load_cals(filepath)

In [30]:
CTD.get_uid('*lookup*.csv','../..')

Exception: Instrument mapped to more than one asset uid.

In [31]:
len(lookup[CTD.serial])

18

In [16]:
CTD.coefficients

{'CC_a0': '1.239929e-003',
 'CC_a1': '2.769904e-004',
 'CC_a2': '-1.321497e-006',
 'CC_a3': '1.885808e-007',
 'CC_g': '-9.813059e-001',
 'CC_h': '1.527103e-001',
 'CC_i': '-4.202770e-004',
 'CC_j': '5.446040e-005',
 'CC_ctcor': '3.250000e-006',
 'CC_cpcor': '-9.570000e-008',
 'CC_C1': '-4.643779e+003',
 'CC_C2': '-7.540100e-002',
 'CC_C3': '8.852730e-004',
 'CC_D1': '6.781200e-002',
 'CC_D2': '0.000000e+000',
 'CC_T1': '3.005444e+001',
 'CC_T2': '-3.892070e-004',
 'CC_T3': '2.670250e-006',
 'CC_T4': '1.718780e-009',
 'CC_T5': '0.000000e+000'}

In [17]:
CTD.date

'20130101'

In [18]:
CTD.serial

'16-7249'

In [19]:
CTD.type

'CTD'

In [ ]:
#csv_file_path = 'C:/Users/areed/Documents/OOI-CGSN/GitHub/OOI-Integration/asset-management/bulk/sensor_bulk_load-AssetRecord.csv'

In [ ]:
os.path.abspath('.')

In [ ]:
os.path.realpath('.')

In [ ]:
os.getcwd()

In [ ]:
!ls

In [23]:
lookup = {}
with open(result) as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        key = row['Serial Number']
        value = row['ASSET_UID']
        lookup[key] = value

In [24]:
lookup

{'120': '#ATAPL-69140-001-120-2',
 '138': 'A00835',
 '3400': 'A00838',
 'S/N 1': 'A01061',
 'S/N 2': 'A01062',
 'S/N 3': 'A01073',
 'TAS141004': 'A01085',
 'TAS141005': 'A01086',
 'TAS141009': 'A01088',
 'P21893-1': 'A01330',
 'P28193-3': 'A01331',
 'P28193-2': 'A01347',
 'P27135-1': 'A01348',
 'P27135-2': 'A01349',
 'P27135-3': 'A01350',
 'P27135-4': 'A01351',
 'P27135-5': 'A01352',
 'P27135-6': 'A01353',
 'P27135-7': 'A01354',
 'P27135-8': 'A01355',
 'P27135-9': 'A01356',
 'P27135-10': 'A01357',
 '28194-1': 'A01359',
 '28194-2': 'A01360',
 '28194-3': 'A01361',
 '28194-4': 'A01362',
 '28194-5': 'A01363',
 '28194-6': 'A01364',
 '28194-7': 'A01365',
 '28194-8': 'A01366',
 'TAS143401': 'A01388',
 '28484-1': 'A01634',
 '28484-2': 'A01635',
 '18478': 'ATAPL-58315-00001',
 '18974': 'ATAPL-58315-00002',
 '18980': 'ATAPL-58315-00003',
 '23338': 'ATAPL-58315-00004',
 '23339': 'ATAPL-58315-00005',
 '3': 'OL000306',
 '4': 'OL000416',
 '5': 'OL000572',
 '7': 'CGCON-NDCL27-00007',
 '8': 'CGCON-NDC

In [25]:
lookup[CTD.serial]

'CGINS-CTDBPO-07249'

In [20]:
# Create a function to find the look_up id
def get_uid_lookup(pattern, path):
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name.lower(), pattern):
                return os.path.join(root, name)

In [21]:
result = get_uid_lookup('*uid*.csv','../..')

In [22]:
result

'../../OOI-CGSN/QAQC_Sandbox/Instrument_UID_Lookup.csv'

In [ ]:
result

In [ ]:
result = get_uid_lookup('instrument_UID_lookup.csv','../..')

In [ ]:
for file in os.listdir('.'):
    if fnmatch.fnmatch(file.lower(), '*uid*.csv'):
        print(file)

In [ ]:
os.getcwd()

In [ ]:
for root, dirs, files in os.walk('../..'):
    for name in files:
        if fnmatch.fnmatch(name.lower(),'*uid*.csv'):
            result = os.path.join(root, name)

In [ ]:
result

In [ ]:
lookup[CTD.serial]

In [ ]:
import os, fnmatch

In [ ]:
for root, dirs, files in os.walk("../.."):
    print(root + ' ' + dirs + ' ' + files)

In [ ]:
def find_lookup(pattern, path):
    result = []
    for root, dirs, files in os.walk('../..'):
        for name in files:
            if fnmatch.fnmatch(name.lower(), pattern.lower()):
                result.append(os.path.join(root, name))
    return result

In [ ]:
find_lookup('instrument_UID_lookup.csv','../..')

In [ ]:
os.path.splitdrive(os.getcwd())

In [ ]:
dirs

In [ ]:
files